In [ ]:
#default_exp losses

In [3]:
#hide
#export
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
from fastai.utils.mem import *
from fastai.vision.gan import *
from PIL import Image

import numpy as np

import torch
import torch.nn.functional as F
import torch.nn as nn

from torch.utils.data import DataLoader
from torch.utils.data.dataset import TensorDataset
import pdb


# Losses

In [4]:
#export
def mse(val, targ):
        return 0.5 * torch.mean((val-targ)**2)

In [5]:
#export
def cov(m):
    fact = 1.0 / (m.size(1) - 1)
    m -= torch.mean(m, dim=1, keepdim=True)
    mt = m.t() 
    return fact * m.matmul(mt).squeeze()

In [6]:
#export
def one_norm(p1, p2):
    return (p1-p2).abs().sum()

In [7]:
#export
def get_adv_loss(preds):
    loss = 0
    for pred in preds:
        loss += mse(pred, torch.ones(pred.shape))
    return loss

In [8]:
#export
def get_rec_loss(orig, orig2, orig_recon, orig2_recon, cycled_orig, \
                cycled_orig2, one_rand, two_rand, one_rand_recon, two_rand_recon):
    return one_norm(orig, orig_recon) + one_norm(orig2, orig2_recon) + one_norm(orig, cycled_orig) \
                + one_norm(orig, cycled_orig2) + one_norm(one_rand, one_rand_recon) \
                + one_norm(two_rand, two_rand_recon)

In [9]:
#export
def get_latent_loss(l1, l2, l1rec, l2rec):
    return one_norm(l1, l1rec) + one_norm(l2, l2rec)

In [10]:
#export
def get_sdist_loss(orig_style, orig2_style, styles):
    styles = torch.cat([orig_style, orig2_style, *styles])
    
    smeans = styles.mean()
    cov_m = cov(styles)
    cov_diag = torch.diag(cov_m)
    
    return one_norm(smeans, torch.ones(1)) + one_norm(cov_m, torch.eye(cov_m.shape[0])) \
            + one_norm(cov_diag, torch.ones(cov_diag.shape))

In [11]:
#export
bce = nn.BCELoss()
def get_seg_loss(orig_seg, orig2_seg, one2two_seg, two2one_seg, one_rand_seg, two_rand_seg):
    orig, orig2 = orig_seg.detach(), orig2_seg.detach()
    return bce(one2two_seg, orig) + bce(two2one_seg, orig2) + bce(one_rand_seg, orig) \
            + bce(two_rand_seg, orig2)

In [13]:
#export
class HiDTLoss(GANModule):
    "Loss function wrapper"
    def __init__(self, loss_funcG:Callable, loss_funcC:Callable, model):
        super().__init__()
        self.loss_funcG,self.loss_funcC,self.model = loss_funcG,loss_funcC,model
        self.content_encoder, self.style_encoder, self.decoder, self.disc = model.content_encoder, model.style_encoder, model.decoder, model.discriminator
        self.style_cap = 16
        self.recent_styles = []
        
        self.lambda1, self.lambda2, self.lambda3, self.lambda4, self.lambda5, self.lambda6, self.lambda7 = 5, 2, 3, 1, 0.1, 4, 7
        
    def forward(self, *args):
        if self.gen_mode:
            return self.generator(*args)
        else:
            return self.discriminator(*args)
        
    def generator(self, output, *args):
        orig, orig2, orig_recon, orig2_recon, orig_style, orig2_style, orig_cont, orig2_cont, \
        one2two, two2one, one2two_style, two2one_style, one2two_cont, two2one_cont, cycled_orig, \
        cycled_orig2, one_rand, two_rand, one_rand_cont, two_rand_cont, one_rand_style, \
        two_rand_style, one_rand_recon, two_rand_recon, rand_style, one2twoc, two2onec, one_randc, \
        two_randc, orig_seg, orig2_seg, one2two_seg, two2one_seg, one_rand_seg, two_rand_seg = output
        
        #adversarial losses
        adv_loss = get_adv_loss([one2two, two2one, one_rand, two_rand, one2twoc, two2onec, one_randc, \
                    two_randc])
        
        #recreation losses
        rec_loss = get_rec_loss(orig, orig2, orig_recon, orig2_recon, cycled_orig, \
                                cycled_orig2, one_rand, two_rand, one_rand_recon, two_rand_recon)
        
        #TODO: pretrain segmenter on dataset
        seg_loss = get_seg_loss(orig_seg, orig2_seg, one2two_seg, two2one_seg, one_rand_seg, two_rand_seg)
        
        #latent code losses
        lat_style_loss = get_latent_loss(orig_style, orig2_style, one2two_style, two2one_style)
        lat_style_rand_loss =  get_latent_loss(rand_style, rand_style, one_rand_style, two_rand_style)
        lat_cont_loss = get_latent_loss(orig_cont, orig2_cont, one2two_cont, two2one_cont)
        lat_cont_loss += get_latent_loss(orig_cont, orig2_cont, one_rand_cont, two_rand_cont)
        
        #style distribution loss
        sdist_loss = get_sdist_loss(orig_style, orig2_style, self.recent_styles)
        
        self.recent_styles.extend([orig_style.data, orig2_style.data])
        
        if len(self.recent_styles) >= self.style_cap:
            self.recent_styles = self.recent_styles[2:]
        
        return self.lambda1*adv_loss + self.lambda2*rec_loss + self.lambda3*seg_loss \
                    + self.lambda4*lat_cont_loss \
                    + self.lambda5*lat_style_loss + self.lambda6*lat_style_rand_loss \
                    + self.lambda7 * sdist_loss

    def discriminator(self, output, *args):
        orig, orig2, one2two, two2one, one2twoc, two2onec, origc, orig2c = output
        
        return mse(orig, torch.ones(orig.shape)) + mse(orig2, torch.ones(orig2.shape)) \
                + mse(one2two, torch.zeros(one2two.shape)) + mse(orig, torch.zeros(two2one.shape)) \
                + mse(one2twoc, torch.zeros(one2twoc.shape)) + mse(two2onec, torch.zeros(two2onec.shape)) \
                + mse(origc, torch.ones(origc.shape)) + mse(orig2c, torch.ones(orig2c.shape)) 
    
        